In [2]:
import sqlalchemy as sqla
import traceback
import datetime
import time
import requests
import json
from pprint import pprint

Station data

In [3]:
APIKEY="dc37139a1f5e7dbb1220eb237bc420120b3381f4"
NAME="Dublin"
STATIONS_URL="https://api.jcdecaux.com/vls/v1/stations" 

r=requests.get(STATIONS_URL,params={"apiKey":APIKEY,"contract":NAME})
stations=json.loads(r.text)

In [4]:
for station in stations:
    print(station['position']['lat'])

53.349562
53.3537415547453
53.336021
53.359405
53.336597
53.33796
53.343368
53.334123
53.344304
53.338755
53.347777
53.336074
53.330091
53.350929
53.341515
53.344603
53.340927
53.348279
53.35893
53.357841
53.344115
53.343893
53.347477
53.339005
53.344153
53.334295
53.340803
53.351464
53.333653
53.343105
53.341428
53.338614
53.341833
53.346637
53.343034
53.346867
53.3547
53.33403
53.346026
53.330662
53.342113
53.357043
53.348875
53.342081
53.346874
53.347932
53.334432
53.343565
53.359624
53.354929
53.350974
53.337132
53.339218
53.347122
53.355173
53.350291
53.354845
53.335742
53.343653
53.347884
53.341288
53.347508
53.341805
53.341645
53.342296
53.337757
53.358115
53.342638
53.339434
53.339334
53.337494
53.35561
53.343897
53.346985
53.355954
53.337824
53.346603
53.345922
53.35023
53.356307
53.359246
53.330362
53.359967
53.356769
53.343456
53.35463
53.356717
53.339764
53.34744
53.355473
53.347972
53.347692
53.344007
53.347106
53.352149
53.345203
53.353331
53.338776
53.359157
53.349013
53

In [9]:
WEATHER_URL="https://api.open-meteo.com/v1/forecast"
HOURLY="temperature_2m","precipitation_probability","weathercode","windspeed_10m"
TIMEZONE="Europe/London"
DAILY="weathercode","temperature_2m_max","temperature_2m_min"
weather_station = {}

for station in stations:
    LATITUDE = station['position']['lat']
    LONGITUDE = station['position']['lng']
    weather=requests.get(WEATHER_URL,params={"latitude":LATITUDE,"longitude":LONGITUDE,"hourly":HOURLY,
    "daily":DAILY,"current_weather":"true","timeformat":"unixtime","timezone":TIMEZONE})
    weather_dict=json.loads(weather.text)
    weather_station[station['number']]=weather_dict
# pprint(weather_dict)


In [26]:
from sqlalchemy import create_engine
import glob
import os
from pprint import pprint
import simplejson as json
import requests
from IPython.display import display

URI = "dbbikes.c06rsktpo8sk.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
USER = "minhly"
PASSWORD = "22201371"
engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)


In [32]:
def weather_to_db():
    for station in weather_station:
        
         # Getting current weather data once every hour: 
        last_update = weather_station.get(station).get('current_weather').get('time')
        temperature = weather_station.get(station).get('current_weather').get('temperature')
        weathercode = weather_station.get(station).get('current_weather').get('weathercode')
        windspeed = weather_station.get(station).get('current_weather').get('windspeed')

        # Getting hourly weather forecast data for the next 12 hours only:
        hourly_time_full = weather_station.get(station).get('hourly').get('time')
        hourly_time = []
        hourly_temp = []
        hourly_preci = []
        hourly_weathercode = []
        hourly_windspeed = []
        i = hourly_time_full.index(last_update)

        for j in range(i,i+12):
            hourly_time.append(hourly_full[j])
            hourly_temp.append(weather_station.get(station).get('hourly').get('temperature_2m')[j])
            hourly_preci.append(weather_station.get(station).get('hourly').get('precipitation_probability')[j])
            hourly_weathercode.append(weather_station.get(station).get('hourly').get('weathercode')[j])
            hourly_windspeed.append(weather_station.get(station).get('hourly').get('windspeed_10m')[j])
        
        # Geting weather forecast data for 7 days
        daily_time = weather_station.get(station).get('daily').get('time')
        daily_weathercode = weather_station.get(station).get('daily').get('weathercode')
        daily_temp_max=weather_station.get(station).get('daily').get('temperature_2m_max')
        daily_temp_min=weather_station.get(station).get('daily').get('temperature_2m_min')
        
        # Insert weather data to table "weather"        
        val = (station, last_update, temperature, weathercode, windspeed, 
        str(hourly_time), str(hourly_temp), str(hourly_preci), str(hourly_weathercode), str(hourly_windspeed)
        str(daily_time),str(daily_weathercode),str(daily_temp_max),str(daily_temp_min))
        engine.execute("insert into weather values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",val)
        break
    return

weather_to_db()

[5.3, 6.7, 1.8, 2.1, 2.9, 0.7, -0.4]


# BELOW IS TESTING CODE

In [10]:
print(weather_station)

{42: {'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 8.638978004455566, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'current_weather': {'temperature': 7.8, 'windspeed': 17.6, 'winddirection': 47.0, 'weathercode': 2, 'time': 1677693600}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 16777

In [28]:
print(weather_station[42])

{'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 8.638978004455566, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'current_weather': {'temperature': 7.8, 'windspeed': 17.6, 'winddirection': 47.0, 'weathercode': 2, 'time': 1677693600}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600

In [11]:
print(weather_dict['current_weather'])

{'temperature': 6.1, 'windspeed': 11.8, 'winddirection': 20.0, 'weathercode': 3, 'time': 1677621600}


In [13]:
print(weather_dict['hourly']['time'])

[1677542400, 1677546000, 1677549600, 1677553200, 1677556800, 1677560400, 1677564000, 1677567600, 1677571200, 1677574800, 1677578400, 1677582000, 1677585600, 1677589200, 1677592800, 1677596400, 1677600000, 1677603600, 1677607200, 1677610800, 1677614400, 1677618000, 1677621600, 1677625200, 1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600, 1677787200, 1677790800, 1677794400, 1677798000, 1677801600, 1677805200, 1677808800, 1677812400, 1677816000, 1677819600, 1677823200, 1677826800, 1677830400, 1677834000, 1677837600, 167

In [14]:
print(weather_dict['hourly']['temperature_2m'])

[7.0, 7.4, 7.7, 7.1, 6.7, 5.2, 6.1, 7.1, 6.6, 7.0, 7.9, 8.0, 7.7, 7.9, 8.3, 8.2, 8.0, 7.5, 7.2, 6.6, 6.5, 6.2, 6.1, 6.8, 7.4, 7.2, 7.1, 7.2, 7.0, 6.9, 6.7, 7.4, 6.9, 7.0, 7.4, 7.3, 8.0, 7.8, 7.1, 7.4, 7.7, 7.5, 7.3, 6.9, 7.1, 6.9, 7.0, 7.3, 7.2, 7.3, 7.3, 7.2, 7.1, 7.2, 6.8, 4.4, 4.2, 5.3, 6.7, 6.9, 7.0, 7.1, 7.6, 7.7, 7.4, 6.9, 5.7, 4.5, 3.9, 3.6, 3.9, 3.5, 2.8, 2.7, 3.1, 2.8, 2.1, 2.5, 3.0, 3.3, 3.5, 4.4, 5.1, 6.0, 7.1, 7.8, 8.3, 8.6, 8.2, 7.5, 6.7, 5.5, 4.4, 3.4, 2.7, 2.6, 3.0, 2.8, 2.9, 3.0, 3.1, 3.6, 3.8, 3.5, 3.5, 3.9, 4.8, 6.1, 7.5, 7.9, 8.0, 7.9, 7.7, 7.5, 7.0, 6.4, 5.8, 4.8, 3.8, 2.8, 1.7, 1.3, 1.2, 1.1, 1.0, 1.0, 1.1, 1.2, 1.4, 2.2, 3.5, 5.1, 7.0, 7.7, 8.1, 8.2, 8.0, 7.6, 7.1, 5.3, 4.4, 3.5, 3.4, 3.6, 3.9, 4.0, 4.1, 4.2, 4.2, 4.2, 4.1, 3.9, 3.8, 3.8, 4.2, 4.7, 5.5, 5.9, 6.3, 6.5, 6.2, 5.7, 4.8, 4.1, 3.2, 2.2, 1.6, 1.1]


In [18]:
string= '[1677542400, 1677546000, 1677549600, 1677553200, 1677556800, 1677560400, 1677564000, 1677567600, 1677571200, 1677574800, 1677578400, 1677582000, 1677585600, 1677589200, 1677592800, 1677596400, 1677600000, 1677603600, 1677607200, 1677610800, 1677614400, 1677618000, 1677621600, 1677625200, 1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600, 1677787200, 1677790800, 1677794400, 1677798000, 1677801600, 1677805200, 1677808800, 1677812400, 1677816000, 1677819600, 1677823200, 1677826800, 1677830400, 1677834000, 1677837600, 1677841200, 1677844800, 1677848400, 1677852000, 1677855600, 1677859200, 1677862800, 1677866400, 1677870000, 1677873600, 1677877200, 1677880800, 1677884400, 1677888000, 1677891600, 1677895200, 1677898800, 1677902400, 1677906000, 1677909600, 1677913200, 1677916800, 1677920400, 1677924000, 1677927600, 1677931200, 1677934800, 1677938400, 1677942000, 1677945600, 1677949200, 1677952800, 1677956400, 1677960000, 1677963600, 1677967200, 1677970800, 1677974400, 1677978000, 1677981600, 1677985200, 1677988800, 1677992400, 1677996000, 1677999600, 1678003200, 1678006800, 1678010400, 1678014000, 1678017600, 1678021200, 1678024800, 1678028400, 1678032000, 1678035600, 1678039200, 1678042800, 1678046400, 1678050000, 1678053600, 1678057200, 1678060800, 1678064400, 1678068000, 1678071600, 1678075200, 1678078800, 1678082400, 1678086000, 1678089600, 1678093200, 1678096800, 1678100400, 1678104000, 1678107600, 1678111200, 1678114800, 1678118400, 1678122000, 1678125600, 1678129200, 1678132800, 1678136400, 1678140000, 1678143600]'
time=json.loads(string)

In [19]:
print(type(string))
print(type(time))

<class 'str'>
<class 'list'>
